# Train Diffusion Model
### Imports

In [9]:
#Set Dir 
import sys, os
sys.path.append(os.path.abspath('..'))

# Torch
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader
import torch.optim as optim
# Utils
import numpy as np
from numpy import ndarray
import logging

# Base Scripts
from Libraries.U_Net import *
from Libraries.Diffusion import *
from Libraries.Utils import *
from Conf import *

### Config
General

In [10]:
logging_level: int = LIGHT_DEBUG #logging.INFO
model_name: str = "diffusion_v1"
model_path: str = f"{MODEL_PATH}/{model_name}.pth"
checkpoint_freq: int = 5 #0 for no checkpoint saving
training_data_name: str = "training_1280"

device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size: int = 2
epochs: int = 100
diffusion_timesteps: int = 500

logging.basicConfig(level=logging_level, format='%(asctime)s - %(levelname)s - %(message)s')
logger: logging.Logger = logging.getLogger(__name__)

U-Net

In [11]:
learning_rate: float = 1e-5
lr_decay: int = 40
lr_gamma: float = 0.1
n_starting_filters: int = 32
n_blocks: int = 2 #Each samples down by factor of 2
n_groups: int = 8 #For group norm
time_embed_dim: int = 128

### Data Loading

In [12]:
file = load_training_data(f"{DATA_PATH}/{training_data_name}.npy")[:4, ...]

2025-02-28 22:27:52,266 - LIGHT_DEBUG - Ndarray loaded from ../Data/training_1280.npy of shape: (1280, 1024, 672)


In [13]:
data_loader = create_dataloader(Audio_Data(file), batch_size)
logger.info(f"Data loaded with shape: {file.shape}")

2025-02-28 22:27:52,458 - INFO - Data loaded with shape: (4, 1024, 672)


### Model Creation
U-Net

In [14]:

u_net = U_NET(in_channels=1, device=device, input_shape=[0, 0, file.shape[-2], file.shape[-1]], n_res_layers=n_blocks, n_starting_filters=n_starting_filters, n_groups=n_groups, time_emb_dim=time_embed_dim).to(device)
if os.path.exists(model_path):
    u_net.load_state_dict(torch.load(model_path, weights_only=False, map_location=device))
    logger.info(f"Model {model_name} loaded with {count_parameters(u_net)} Parameters")
else: 
    logger.info(f"Model {model_name} created with {count_parameters(u_net)} Parameters")

optimizer = optim.AdamW(u_net.parameters(), lr=learning_rate, weight_decay=0.05)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay, gamma=lr_gamma)
u_net = nn.DataParallel(u_net)
u_net = u_net.to(device)

2025-02-28 22:27:52,894 - INFO - Model diffusion_v1 loaded with 833281 Parameters


Diffusion

In [15]:
diffusion = Diffusion(u_net=u_net, u_net_optimizer=optimizer,diffusion_timesteps=diffusion_timesteps)
noise_schedule = cosine_noise(T=diffusion_timesteps)

### Train

In [ ]:
diffusion.train(data_loader, device, epochs=epochs, loss_function=U_Net_loss, noise_schedule=noise_schedule, checkpoint_freq=checkpoint_freq, model_path=model_path)

2025-02-28 22:29:15,000 - LIGHT_DEBUG - Batch 01/02

2025-02-28 22:30:14,547 - INFO - Epoch 01: Avg. Loss: 5.88422e-01 Remaining Time: 03h 53min 31s


2025-02-28 22:30:14,000 - LIGHT_DEBUG - Batch 02/02
2025-02-28 22:31:07,000 - LIGHT_DEBUG - Batch 01/02

2025-02-28 22:32:05,563 - INFO - Epoch 02: Avg. Loss: 5.78866e-01 Remaining Time: 03h 26min 14s


2025-02-28 22:32:05,000 - LIGHT_DEBUG - Batch 02/02
2025-02-28 22:33:02,000 - LIGHT_DEBUG - Batch 01/02

...

2025-02-28 22:57:28,000 - LIGHT_DEBUG - Batch 02/02
2025-02-28 22:58:30,000 - LIGHT_DEBUG - Batch 01/02

2025-02-28 22:59:39,202 - INFO - Epoch 16: Avg. Loss: 5.20425e-01 Remaining Time: 02h 46min 46s


2025-02-28 22:59:39,000 - LIGHT_DEBUG - Batch 02/02
2025-02-28 23:00:37,000 - LIGHT_DEBUG - Batch 01/02

2025-02-28 23:01:29,372 - INFO - Epoch 17: Avg. Loss: 5.14987e-01 Remaining Time: 02h 44min 03s


2025-02-28 23:01:29,000 - LIGHT_DEBUG - Batch 02/02
